# PRÁCTICA 1
## Estrategias de particionado, Naive Bayes y Análisis ROC
### Fundamentos de Aprendizaje Automático
##### Grupo 1461: Gloria del Valle Cano y Eva Gutiérrez Vanmoen

In [207]:
from abc import ABCMeta, abstractmethod
import random

from functools import reduce
import math
import operator

from Datos import Datos
import Clasificador
import EstrategiaParticionado
import numpy as np
import pandas as pd

from sklearn import preprocessing
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix, roc_curve, auc

import matplotlib.pyplot as plt

from IPython.display import HTML, display

### Apartado 1 : Particionado
Análisis de las dos estrategias de particionado propuestas: simple, y
cruzada, para los conjuntos propuestos: lenses, german y tic-tac-toe. El
análisis consiste en una descripción de los índices de train y test
devueltos por cada uno de los métodos de particionado, junto con un
comentario sobre las ventajas/desventajas de cada uno de ellos.

In [174]:
d1 = Datos('./ConjuntosDatos/tic-tac-toe.data')
d2 = Datos('./ConjuntosDatos/german.data')
d3 = Datos('./ConjuntosDatos/lenses.data')

Hemos cargado los conjuntos de datos que vamos a utilizar y a continuación vamos a mostrar la validación simple implementada.
##### VSimple

In [175]:
 def creaParticiones(self,datos,seed=None):
    
    random.seed(seed)

    # K iteraciones = numero de particiones
    for k in range(self.numeroParticiones):

      p = Particion()

      # Obtenemos una array aleatoria de las filas de nuestros datos
      npaux = np.random.permutation(datos.shape[0])

      i = int(len(npaux.tolist())*self.porcentaje) # indice del porcentaje deseado para entrenamiento
      
      # Dividimos datos en entrenamiento y de prueba
      p.indicesTrain = npaux.tolist()[0:i]
      p.indicesTest = npaux.tolist()[i:]

      self.listaParticiones.append(p) # Insertamos la particion creada

Realizamos una partición y seleccionamos al azar el conjunto de entrenamiento y el resto será para el test.
Lo bueno es que es muy fácil de implementar, pero existe el problema de que la división no puede ser suficientemente determinante como para tomar una decisión, es decir, los índices de test que se cogen para probar el entrenamiento pueden no ser determinantes, demasiados, pocos... Debido a la variabilidad y a la división de los datos, no sabremos qué será más óptimo hasta que hagamos más pruebas.
##### Vcruzada

In [176]:
  def creaParticiones(self,datos,seed=None):   
    
    random.seed(seed)

    # Obtenemos una array aleatoria de las filas de nuestros datos
    npaux1 = np.random.permutation(datos.shape[0])

    nL = int(len(npaux1.tolist())) # numero de lineas de datos

    step = int(nL/self.numeroParticiones) # numero de datos por cada subconjunto

    # laux es una lista de la array aleatoria (npaux2) de nuestros indices de los subconjuntos
    npaux2 = np.random.permutation(range(0,nL,step))
    laux = npaux2.tolist()
    
    # Dividimos en K-1 subconjuntos para entrenamiento y el resto test
    for k in range(self.numeroParticiones):
      
      p = Particion()

      # Primero obtenemos el subconjunto para tests
      i = laux[0] # el indice inicial de nuestros datos de test
      p.indicesTest = npaux1.tolist()[i:i+step]
      laux.pop(0) # eliminamos el indice de bloque ya usado para test

      # K-1 subconjuntos para datos de entrenamiento
      for j in npaux2.tolist():
        if j != i:
          # Si tratamos con el ultimo bloque de datos
          if j == max(npaux2.tolist()):
            p.indicesTrain.extend(npaux1.tolist()[j:])
          else:
            p.indicesTrain.extend(npaux1.tolist()[j:j+step])

      # Insertamos la particion creada de cada iteracion
      self.listaParticiones.append(p)

En este caso, el entrenamiento se crea con nFolds - 1 (nFolds = n divisiones) y el de test con la restante. Es una de las maneras más completas de probar los datos, pero es más costoso a nivel computacional.
###### Indices devueltos por tic-tac-toe.data

In [177]:
NB = Clasificador.ClasificadorNaiveBayes()
print ("VALIDACION SIMPLE (tic-tac-toe.data): ")
v_simple = EstrategiaParticionado.ValidacionSimple('ValidacionSimple',3,0.8)
p = v_simple.creaParticiones(d1.datos)
for k in p:
	print ("Indices Test: ")
	print (k.indicesTest)
	print ("Indices Train: ")
	print (k.indicesTrain)

print ("VALIDACION CRUZADA (tic-tac-toe.data): ")
v_cross = EstrategiaParticionado.ValidacionCruzada('ValidacionCruzada',5)
p = v_cross.creaParticiones(d1.datos)
for k in p:
	print ("Indices Test: ")
	print (k.indicesTest)
	print ("Indices Train: ")
	print (k.indicesTrain)
	print ("----")
print ("-----------------------")

VALIDACION SIMPLE (tic-tac-toe.data): 
Indices Test: 
[800, 542, 886, 449, 224, 225, 360, 120, 201, 128, 762, 594, 267, 705, 464, 349, 169, 944, 774, 271, 789, 175, 233, 495, 46, 823, 516, 58, 572, 456, 608, 799, 934, 91, 847, 612, 311, 873, 747, 82, 630, 864, 42, 890, 763, 27, 739, 743, 497, 909, 778, 915, 177, 693, 832, 729, 83, 450, 365, 223, 677, 315, 952, 575, 420, 297, 85, 379, 596, 253, 622, 829, 830, 831, 935, 189, 77, 154, 827, 657, 422, 522, 190, 653, 591, 17, 30, 172, 197, 329, 627, 628, 599, 679, 690, 276, 98, 528, 75, 635, 33, 756, 735, 101, 340, 52, 13, 454, 893, 117, 385, 699, 930, 165, 166, 433, 487, 824, 37, 485, 680, 374, 682, 919, 241, 558, 319, 198, 723, 625, 618, 231, 188, 140, 396, 483, 781, 570, 149, 357, 631, 470, 476, 716, 87, 10, 955, 455, 314, 868, 883, 21, 109, 927, 931, 803, 587, 111, 943, 718, 532, 353, 51, 585, 432, 825, 221, 489, 53, 86, 904, 119, 702, 147, 453, 168, 207, 557, 685, 226, 307, 804, 458, 863, 821, 880, 328, 672, 284, 540, 583, 758]
Indices 

##### Indices devueltos por german.data

In [178]:
NB = Clasificador.ClasificadorNaiveBayes()
print ("VALIDACION SIMPLE (german.data): ")
v_simple = EstrategiaParticionado.ValidacionSimple('ValidacionSimple',3,0.8)
p = v_simple.creaParticiones(d2.datos)
for k in p:
	print ("Indices Test: ")
	print (k.indicesTest)
	print ("Indices Train: ")
	print (k.indicesTrain)

print ("VALIDACION CRUZADA (german.data): ")
v_cross = EstrategiaParticionado.ValidacionCruzada('ValidacionCruzada',5)
p = v_cross.creaParticiones(d2.datos)
for k in p:
	print ("Indices Test: ")
	print (k.indicesTest)
	print ("Indices Train: ")
	print (k.indicesTrain)
	print ("----")
print ("-----------------------")

VALIDACION SIMPLE (german.data): 
Indices Test: 
[151, 556, 730, 595, 856, 257, 603, 249, 991, 741, 726, 731, 466, 594, 635, 864, 665, 33, 855, 580, 578, 828, 17, 105, 483, 25, 32, 110, 984, 588, 226, 207, 937, 116, 812, 476, 172, 182, 99, 208, 939, 284, 905, 504, 627, 211, 789, 844, 270, 36, 600, 77, 618, 616, 640, 217, 518, 909, 662, 193, 438, 868, 129, 305, 489, 687, 310, 31, 201, 50, 774, 762, 147, 875, 865, 645, 374, 617, 733, 533, 880, 638, 853, 433, 228, 849, 513, 710, 367, 351, 577, 974, 647, 999, 964, 113, 587, 602, 516, 81, 256, 335, 164, 427, 677, 311, 802, 737, 14, 585, 58, 133, 928, 340, 611, 27, 967, 787, 753, 19, 525, 904, 612, 231, 278, 461, 549, 987, 806, 957, 724, 469, 657, 344, 107, 155, 170, 628, 722, 263, 15, 667, 297, 313, 663, 41, 526, 544, 715, 3, 961, 260, 477, 581, 601, 45, 397, 261, 157, 234, 676, 583, 943, 98, 331, 576, 392, 37, 357, 653, 140, 966, 358, 275, 894, 596, 60, 47, 386, 39, 393, 572, 161, 982, 462, 78, 831, 670, 995, 361, 669, 173, 128, 555, 206, 

##### Indices devueltos por lenses.data

In [179]:
NB = Clasificador.ClasificadorNaiveBayes()
print ("VALIDACION SIMPLE (lenses.data): ")
v_simple = EstrategiaParticionado.ValidacionSimple('ValidacionSimple',3,0.8)
p = v_simple.creaParticiones(d3.datos)
for k in p:
	print ("Indices Test: ")
	print (k.indicesTest)
	print ("Indices Train: ")
	print (k.indicesTrain)

print ("VALIDACION CRUZADA (lenses.data): ")
v_cross = EstrategiaParticionado.ValidacionCruzada('ValidacionCruzada',5)
p = v_cross.creaParticiones(d3.datos)
for k in p:
	print ("Indices Test: ")
	print (k.indicesTest)
	print ("Indices Train: ")
	print (k.indicesTrain)
	print ("----")
print ("-----------------------")

VALIDACION SIMPLE (lenses.data): 
Indices Test: 
[9, 5, 21, 11, 8]
Indices Train: 
[17, 2, 20, 4, 10, 14, 1, 7, 18, 23, 0, 22, 19, 3, 15, 16, 13, 6, 12]
Indices Test: 
[18, 1, 13, 7, 22]
Indices Train: 
[4, 16, 2, 9, 15, 0, 12, 21, 19, 10, 8, 20, 11, 17, 23, 3, 14, 5, 6]
Indices Test: 
[19, 1, 23, 21, 0]
Indices Train: 
[14, 13, 18, 10, 8, 2, 7, 6, 5, 15, 9, 12, 16, 3, 20, 17, 11, 4, 22]
VALIDACION CRUZADA (lenses.data): 
Indices Test: 
[13, 11, 9, 0]
Indices Train: 
[12, 8, 3, 7, 10, 18, 6, 22, 2, 21, 20, 1, 19, 23, 15, 4, 5, 16, 17, 14]
----
Indices Test: 
[12, 8, 3, 7]
Indices Train: 
[13, 11, 9, 0, 10, 18, 6, 22, 2, 21, 20, 1, 19, 23, 15, 4, 5, 16, 17, 14]
----
Indices Test: 
[10, 18, 6, 22]
Indices Train: 
[13, 11, 9, 0, 12, 8, 3, 7, 2, 21, 20, 1, 19, 23, 15, 4, 5, 16, 17, 14]
----
Indices Test: 
[2, 21, 20, 1]
Indices Train: 
[13, 11, 9, 0, 12, 8, 3, 7, 10, 18, 6, 22, 19, 23, 15, 4, 5, 16, 17, 14]
----
Indices Test: 
[19, 23, 15, 4]
Indices Train: 
[13, 11, 9, 0, 12, 8, 3, 7, 10,

### Apartado 2: Naive-Bayes
Tabla con los resultados de la ejecución para los conjuntos de datos
analizados (lenses, tic-tac-toe y german). Considerar los dos tipos de
particionado.
Los resultados se refieren a las tasas de error/acierto y deben incluirse
tanto con la corrección de Laplace como sin ella. Se debe incluir tanto
el promedio de error para las diferentes particiones como su desviación
típica. Es importante mostrar todos los resultados agrupados en una 
tabla para facilitar su evaluación.
Breve análisis de los resultados anteriores

#### Test con tic-tac-toe.data

Probamos primero Validación Simple sin laPlace

In [180]:
estrategia = EstrategiaParticionado.ValidacionSimple('ValidacionSimple',5,0.8)
clasificadorNB = Clasificador.ClasificadorNaiveBayes()
MSE11 = clasificadorNB.validacion(estrategia, d1, clasificadorNB)
print(MSE11)

[0.375]


Probamos de la misma manera con la correccion de laPlace

In [181]:
estrategia = EstrategiaParticionado.ValidacionSimple('ValidacionSimple',5,0.8)
clasificadorNB = Clasificador.ClasificadorNaiveBayes()
MSE12 = clasificadorNB.validacion(estrategia, d1, clasificadorNB,laPlace=True)
print(MSE12)

[0.6614583333333334]


Ahora con Validación Cruzada sin laPlace

In [182]:
estrategia = EstrategiaParticionado.ValidacionCruzada('ValidacionCruzada',5)
clasificadorNB = Clasificador.ClasificadorNaiveBayes()
MSE = clasificadorNB.validacion(estrategia, d1, clasificadorNB)
print("Lista de MSE:")
for i in MSE:
    print (i)
print("\nMedia MSE: ")
m11 = np.mean(MSE)
print(m11)
print("\nDesviación típica: ")
dt11 = np.std(MSE)
print(dt11)

Lista de MSE:
0.6701570680628273
0.33507853403141363
0.0
0.3612565445026178
0.3612565445026178

Media MSE: 
0.34554973821989526

Desviación típica: 
0.21230995917100026


Probamos de la misma manera con la correccion de laPlace

In [198]:
estrategia = EstrategiaParticionado.ValidacionCruzada('ValidacionCruzada',5)
clasificadorNB = Clasificador.ClasificadorNaiveBayes()
MSE = clasificadorNB.validacion(estrategia, d1, clasificadorNB, laPlace=True)
print("Lista de MSE:")
for i in MSE:
    print (i)
print("\nMedia MSE: ")
m12 = np.mean(MSE)
print(m12)
print("\nDesviación típica: ")
dt12 = np.std(MSE)
print(dt12)

Lista de MSE:
0.36649214659685864
0.3333333333333333
0.6178010471204188
0.6701570680628273
0.675392670157068

Media MSE: 
0.5326352530541012

Desviación típica: 
0.15091052266014932


#### Test con german.data

Probamos primero Validación Simple sin laPlace

In [184]:
estrategia = EstrategiaParticionado.ValidacionSimple('ValidacionSimple',5,0.8)
clasificadorNB = Clasificador.ClasificadorNaiveBayes()
MSE21 = clasificadorNB.validacion(estrategia, d2, clasificadorNB)
print(MSE21)

KeyError: None

Probamos de la misma manera con la correccion de laPlace

In [185]:
estrategia = EstrategiaParticionado.ValidacionSimple('ValidacionSimple',5,0.8)
clasificadorNB = Clasificador.ClasificadorNaiveBayes()
MSE22 = clasificadorNB.validacion(estrategia, d2, clasificadorNB,laPlace=True)
print(MSE22)

KeyError: None

Ahora con Validación Cruzada sin laPlace

In [122]:
estrategia = EstrategiaParticionado.ValidacionCruzada('ValidacionCruzada',5)
clasificadorNB = Clasificador.ClasificadorNaiveBayes()
MSE = clasificadorNB.validacion(estrategia, d2, clasificadorNB)
print("Lista de MSE:")
for i in MSE:
    print (i)
print("\nMedia MSE: ")
m21 = np.mean(MSE)
print(m21)
print("\nDesviación típica: ")
dt21 = np.std(MSE)
print(dt21)

KeyError: None

Probamos de la misma manera con la correccion de laPlace

In [186]:
estrategia = EstrategiaParticionado.ValidacionCruzada('ValidacionCruzada',5)
clasificadorNB = Clasificador.ClasificadorNaiveBayes()
MSE = clasificadorNB.validacion(estrategia, d2, clasificadorNB, laPlace=True)
print("Lista de MSE:")
for i in MSE:
    print (i)
print("\nMedia MSE: ")
m22 = np.mean(MSE)
print(m21)
print("\nDesviación típica: ")
dt22 = np.std(MSE)
print(dt22)

KeyError: None

NO ENTIENDO POR QUE, EN EL MAIN SALE BIEN (ver salida del main)
__________________________________________
SIN LAPLACE:
Error NB simple: 
0.695

Error NB cruzada: 
[0.75, 0.275, 0.345, 0.31, 0.32]

Media errores NB cruzada: 
0.4

__________________________________________
CON LAPLACE:
Error NB simple: 
0.32

Error NB cruzada: 
[0.31, 0.295, 0.29, 0.325, 0.28]

Media errores NB cruzada: 
0.3

In [187]:
MSE21 = 0.695
MSE22 = 0.32
m21 = 0.4
m22 = 0.3
l1 = [0.75, 0.275, 0.345, 0.31, 0.32]
l2 = [0.31, 0.295, 0.29, 0.325, 0.28]
dt21 = np.std(l1)
dt22 = np.std(l2)

#### Test con lenses.data

Probamos primero Validación Simple sin laPlace

In [124]:
estrategia = EstrategiaParticionado.ValidacionSimple('ValidacionSimple',5,0.8)
clasificadorNB = Clasificador.ClasificadorNaiveBayes()
MSE31 = clasificadorNB.validacion(estrategia, d3, clasificadorNB)
print(MSE31)

[0.6]


Probamos de la misma manera con la corrección de laPlace

In [125]:
estrategia = EstrategiaParticionado.ValidacionSimple('ValidacionSimple',5,0.8)
clasificadorNB = Clasificador.ClasificadorNaiveBayes()
MSE32 = clasificadorNB.validacion(estrategia, d3, clasificadorNB, laPlace=True)
print(MSE32)

[0.8]


Ahora con Validación Cruzada sin laPlace

In [139]:
estrategia = EstrategiaParticionado.ValidacionCruzada('ValidacionCruzada',5)
clasificadorNB = Clasificador.ClasificadorNaiveBayes()
MSE = clasificadorNB.validacion(estrategia, d3, clasificadorNB)
print("Lista de MSE:")
for i in MSE:
    print (i)
print("\nMedia MSE: ")
m31 = np.mean(MSE)
print(m31)
print("\nDesviación típica: ")
dt31 = np.std(MSE)
print(dt31)

Lista de MSE:
1.0
0.25
0.5
0.5
0.0

Media MSE: 
0.45

Desviación típica: 
0.33166247903554


Probamos de la misma manera con la corrección de laPlace

In [140]:
estrategia = EstrategiaParticionado.ValidacionCruzada('ValidacionCruzada',5)
clasificadorNB = Clasificador.ClasificadorNaiveBayes()
MSE = clasificadorNB.validacion(estrategia, d3, clasificadorNB, laPlace=True)
print("Lista de MSE:")
for i in MSE:
    print (i)
print("\nMedia MSE: ")
m32 = np.mean(MSE)
print(m32)
print("\nDesviación típica: ")
dt32 = np.std(MSE)
print(dt32)

Lista de MSE:
0.75
1.0
0.0
0.25
0.5

Media MSE: 
0.5

Desviación típica: 
0.3535533905932738


#### Resumen en tablas

In [ ]:
salida = "<table><tr><th>VSimple</th><th>tic-tac-toe.data</th><th>german.data</th><th>lenses.data</th></tr><tr><td>NB sin laPlace</td>"
salida += "<td>%f</td>" % (MSE11[0])

#salida += "<td>%f</td>" % (MSE21[0])
salida += "<td>%f</td>" % (MSE21)

salida += "<td>%f</td></tr>" % (MSE31[0])
salida += "<tr><td>NB con laPlace</td>"
salida += "<td>%f</td>" % (MSE12[0])

#salida += "<td>%f</td>" % (MSE22[0])
salida += "<td>%f</td>" % (MSE22)

salida += "<td>%f</td></tr></table>" % (MSE32[0])

salida2 = "<table><tr><th>VCruzada</th><th>tic-tac-toe.data</th><th>german.data</th><th>lenses.data</th></tr><tr><td>NB sin laPlace</td>"
salida2 += "<td>%.4f ± %.3f</td>" % (m11,dt11)

salida2 += "<td>%.4f ± %.3f</td>" % (m21,dt21)
salida2 += "<td>%.4f ± %.3f</td>" % (m31,dt31)
salida2 += "<tr><td>NB con Laplace</td>"
salida2 += "<td>%.4f ± %.3f</td>" % (m12,dt12)
salida2 += "<td>%.4f ± %.3f</td>" % (m22,dt22)
salida2 += "<td>%.4f ± %.3f</td>" % (m32,dt32)

In [192]:
display(HTML(salida))

VSimple,tic-tac-toe.data,german.data,lenses.data
NB sin laPlace,0.375000,0.695000,0.600000
NB con laPlace,0.661458,0.320000,0.800000


Comentarios?

In [193]:
display(HTML(salida2))

VCruzada,tic-tac-toe.data,german.data,lenses.data
NB sin laPlace,0.3455 ± 0.212,0.4000 ± 0.176,0.4500 ± 0.332
NB con Laplace,0.5927 ± 0.125,0.3000 ± 0.016,0.5000 ± 0.354


Comentarios?

### Apartado 3: Scikit-Learn
Incluir los mismos resultados que en el apartado 2 pero usando los
métodos del paquete scikit-learn. Comparar y analizar los resultados. 

##### La implementación para estos resultados se encuentra en el módulo ParticionadoSK.py


###### Para tic-tac-toe.data
* MSE de NB usando VSimple con laPlace: 0.2956506610995129 con dt: 0.026530565154327092
* MSE de NB usando VCruzada con laPlace: 0.5490605427974948 con dt: 0.11482254697286015
* MSE de NB usando VSimple sin laPlace: 0.2961986778009742 con dt: 0.026569633972322836
* MSE de NB usando VCruzada sin laPlace: 0.5490605427974948 con dt: 0.11482254697286015

###### Para german.data
* MSE de NB usando VSimple con laPlace: 0.3649080000000001 con dt: 0.02825695553310726
* MSE de NB usando VCruzada con laPlace: 0.361 con dt: 0.025000000000000022
* MSE de NB usando VSimple sin laPlace: 0.364556 con dt: 0.028259845434821487
* MSE de NB usando VCruzada sin laPlace: 0.362 con dt: 0.02400000000000002

###### Para lenses.data
* MSE de NB usando VSimple con laPlace: 0.3819444444444444 con dt: 0.16996164236833913
* MSE de NB usando VCruzada con laPlace: 0.25174825174825166 con dt: 0.02097902097902099
* MSE de NB usando VSimple sin laPlace: 0.35416666666666663 con dt: 0.18827002377397167
* MSE de NB usando VCruzada sin laPlace: 0.25174825174825166 con dt: 0.02097902097902099

Podemos ver que la implementación de Scikit sigue siendo más óptima que la propuesta, pero se acercan bastante y nos ha servido para entender el funcionamiento y la importancia de la validación.

Repetimos el código (un poco más simplificado) más abajo a modo de pseudocódigo:

### Apartado 4: Evaluación de hipótesis mediante Análisis ROC
Matriz de confusión y diagramas del clasificador en el espacio ROC

#### Matriz de confusión: tic-tac-toe.data

In [202]:
actual = pd.Series([MSE11[0], MSE12[0]], name='Actual')
pred = pd.Series([m11, m12], name='Prediccion')
df_confusion = pd.crosstab(actual,pred)
print (df_confusion)

Prediccion  0.345550  0.532635
Actual                        
0.375000           1         0
0.661458           0         1


In [205]:
actual = pd.Series([MSE31[0], MSE32[0]], name='Actual')
pred = pd.Series([m31, m32], name='Prediccion')
df_confusion = pd.crosstab(actual,pred)
print (df_confusion)

Prediccion  0.45  0.50
Actual                
0.6            1     0
0.8            0     1


In [ ]:
#Falta la de german

In [208]:
#Falta grafica ROC, no se si se hace asi
fpr = dict()
tpr = dict()
roc_auc = dict()
for i in range(pred.shape[0]):
    fpr[i], tpr[i] = roc_curve(pred.ravel(), actual.ravel())
    roc_auc[i] = auc(fpr[i], tpr[i])
    
plt.figure()
plt.plot(fpr[2], tpr[2], color='darkorange', lw=2, label='Curva ROC (area=%0.2f)' %roc_auc[2])
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel("FPR")
plt.ylabel("TPR")
plt.title("Resultado ROC")
plt.legend(loc='lower right')
plt.show()

ValueError: continuous format is not supported